# Summary

Link to paper: http://www.sciencedirect.com/science/article/pii/S0092867415004304


This notebook is the same as `taipale.ipynb`, but it uses an older method to map RefSeq mutations to UniProt.

----

# Imports

In [1]:
%run imports.ipynb

2016-07-27 01:19:57.217220


In [2]:
NOTEBOOK_NAME = 'taipale_tr'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

os.environ['NOTEBOOK_NAME'] = NOTEBOOK_NAME
os.environ['DB_PORT'] = '8307'

In [3]:
%run mysqld.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-07-27 01:19:57.367509


In [4]:
db_remote = datapkg.MySQL(
    connection_string=os.environ['DATAPKG_CONNECTION_STRING'] + '/staging', 
    shared_folder=os.environ['NOTEBOOK_NAME'], 
    storage_host=None, 
    echo=False, 
    db_engine='InnoDB'
)

# Load mutation data

In [5]:
ls ../downloads/taipale/

1-s2.0-S0092867415004304-main.pdf  mmc2.xlsx  mmc4.xlsx  mmc6.xlsx  mmc8.pdf
mmc1.xlsx                          mmc3.xlsx  mmc5.xlsx  mmc7.xlsx


In [6]:
mutation_df = pd.read_excel('../downloads/taipale/mmc1.xlsx')

In [7]:
display(mutation_df.head(2))
print(mutation_df.shape[0])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,HGMD_accession,HGMD_variant_class,dbSNP_ID,Disease
0,Disease mutation,A2M,2,2_18118,NM_000014:c.2915G>A,NP_000005:p.C972Y,CM920001,DM,rs1800433,Chronic obstructive pulmonary disease
1,Disease mutation,A2M,2,2_18119,NM_000014:c.2998G>A,NP_000005:p.V1000I,CM980001,DP,rs669,"Alzheimer disease, association with"


2960


### mutation_df_1

In [8]:
mutation_df['refseq_id'] = mutation_df['Mutation_RefSeq_AA'].apply(lambda x: x.split(':')[0])
mutation_df['refseq_mutation'] = mutation_df['Mutation_RefSeq_AA'].apply(lambda x: x.split('.')[-1])

In [9]:
display(mutation_df.head(2))
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,HGMD_accession,HGMD_variant_class,dbSNP_ID,Disease,refseq_id,refseq_mutation
0,Disease mutation,A2M,2,2_18118,NM_000014:c.2915G>A,NP_000005:p.C972Y,CM920001,DM,rs1800433,Chronic obstructive pulmonary disease,NP_000005,C972Y
1,Disease mutation,A2M,2,2_18119,NM_000014:c.2998G>A,NP_000005:p.V1000I,CM980001,DP,rs669,"Alzheimer disease, association with",NP_000005,V1000I


Number of rows: 2960
Number of unique RefSeq nucleotide mutations: 2960
Number of unique RefSeq amino acid mutations: 2958


In [10]:
print("Duplicated RefSeq amino acid mutations:")
duplicates = mutation_df[mutation_df['Mutation_RefSeq_AA'].duplicated()]['Mutation_RefSeq_AA']
mutation_df[mutation_df['Mutation_RefSeq_AA'].isin(duplicates)]

Duplicated RefSeq amino acid mutations:


,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,HGMD_accession,HGMD_variant_class,dbSNP_ID,Disease,refseq_id,refseq_mutation
1404,Disease mutation,HSPB8,26353,26353_7864,NM_014365:c.423G>C,NP_055180:p.K141N,CM041377,DM,rs104894345,"Neuropathy, distal hereditary motor, type II",NP_055180,K141N
1405,Disease mutation,HSPB8,26353,26353_7865,NM_014365:c.423G>T,NP_055180:p.K141N,CM050270,DM,rs104894345,Charcot-Marie-Tooth disease 2L,NP_055180,K141N
1875,Disease mutation,NIPA1,123606,123606_15185,NM_144599:c.316G>A,NP_653200:p.G106R,CM050748,DM,rs104894490,"Spastic paraplegia, autosomal dominant",NP_653200,G106R
1876,Disease mutation,NIPA1,123606,123606_15186,NM_144599:c.316G>C,NP_653200:p.G106R,CM050749,DM,rs104894490,"Spastic paraplegia, autosomal dominant",NP_653200,G106R


In [11]:
mutation_df_1 = mutation_df.copy()

## NCBI to UniProt

Map NCBI to UniProt.

### mutation_df_2 (old)

Use the `uniprot_identifier` table to map Refseq to Uniprot.

In [12]:
mutation_df = mutation_df_1.copy()

In [13]:
if os.path.isfile(op.join(NOTEBOOK_NAME, 'refseq_to_uniprot.pickle')):
    print("Loading data from file...")
    refseq_to_uniprot = pd.read_pickle(op.join(NOTEBOOK_NAME, 'refseq_to_uniprot.pickle'))
else:
    sql_query = """\
    select *
    from uniprot_kb.uniprot_identifier
    join uniprot_kb.uniprot_sequence using (uniprot_id)
    where identifier_type = 'RefSeq'
    and (identifier_id like '{}.%%');
    """.format(".%%' or identifier_id like '".join(set(mutation_df['refseq_id'].values)))
    refseq_to_uniprot = pd.read_sql_query(sql_query, db_remote.engine)
    
    os.makedirs(NOTEBOOK_NAME, exist_ok=True)
    refseq_to_uniprot.to_pickle(op.join(NOTEBOOK_NAME, 'refseq_to_uniprot_2.pickle'))

Loading data from file...


In [14]:
refseq_to_uniprot['refseq_id'] = refseq_to_uniprot['identifier_id'].apply(lambda x: x.split('.')[0])

In [15]:
print("DF mapping Refseq to Uniprot:")
display(refseq_to_uniprot.head(2))
print("Number of rows:", refseq_to_uniprot.shape[0])

DF mapping Refseq to Uniprot:


,uniprot_id,id,identifier_id,identifier_type,db,uniprot_name,protein_name,organism_name,gene_name,protein_existence,sequence_version,uniprot_sequence,refseq_id
0,P01023,61589,NP_000005.2,RefSeq,sp,A2MG_HUMAN,Alpha-2-macroglobulin,Homo sapiens,A2M,1.0,3.0,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,NP_000005
1,A4Z6T7,58633781,NP_000006.2,RefSeq,tr,A4Z6T7_HUMAN,Arylamine N-acetyltransferase 2,Homo sapiens,NAT2,3.0,1.0,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...,NP_000006


Number of rows: 1493


In [16]:
mutation_df = (
    mutation_df.merge(refseq_to_uniprot, on=['refseq_id'])
)

In [17]:
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0], "<--")
print("Number of unique RefSeq / Uniprot nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT', 'uniprot_id']).shape[0])
print("Number of unique RefSeq / Uniprot amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA', 'uniprot_id']).shape[0])

Number of rows: 3902
Number of unique RefSeq nucleotide mutations: 2948
Number of unique RefSeq amino acid mutations: 2946 <--
Number of unique RefSeq / Uniprot nucleotide mutations: 3902
Number of unique RefSeq / Uniprot amino acid mutations: 3900


Lost **12** amino acid mutations because RefSeq does not have a corresponding Uniprot ID

In [18]:
mutation_df_2 = mutation_df.copy()

### mutation_df_failed

In [19]:
mutation_df_failed = (
    mutation_df_1
    .merge(refseq_to_uniprot, on=['refseq_id'], how='left')
)

In [20]:
mutation_df_failed = mutation_df_failed[['Mutation_RefSeq_AA', 'uniprot_id', 'refseq_mutation']]

In [21]:
assert not set(mutation_df_failed['Mutation_RefSeq_AA']) ^ set(mutation_df_1['Mutation_RefSeq_AA'])

In [22]:
mutation_df_failed['unique_id'] = (
    mutation_df_failed['uniprot_id'] + '.' + mutation_df_failed['refseq_mutation']
)

In [23]:
mutation_df_failed['errors'] = ''

In [24]:
mutation_df_failed.loc[mutation_df_failed['uniprot_id'].isnull(), 'errors'] = (
    "Could not map UniParc to UniProt. "
)

In [25]:
mutation_df_failed[mutation_df_failed['errors'] != ''].tail()

,Mutation_RefSeq_AA,uniprot_id,refseq_mutation,unique_id,errors
1932,NP_009296:p.E268K,NaN,E268K,NaN,Could not map UniParc to UniProt.
2063,NP_057685:p.G236R,NaN,G236R,NaN,Could not map UniParc to UniProt.
3310,NP_003973:p.T5I,NaN,T5I,NaN,Could not map UniParc to UniProt.
3311,NP_003973:p.A140P,NaN,A140P,NaN,Could not map UniParc to UniProt.
3312,NP_003973:p.L334R,NaN,L334R,NaN,Could not map UniParc to UniProt.


In [26]:
mutation_df_failed.shape

(3914, 5)

## Mutation matches sequence

Make sure that the AA in UniProt sequence matches the AA in the RefSeq mutation.

### mutation_df_3

In [27]:
mutation_df = mutation_df_2.copy()

In [28]:
mutation_df['mutation_matches_sequence'] = (
    mutation_df[['refseq_mutation', 'uniprot_sequence']]
    .apply(lambda x: ascommon.sequence_tools.mutation_matches_sequence(*x), axis=1)
)
mutation_df = mutation_df[mutation_df['mutation_matches_sequence']]

In [29]:
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0], "<--")
print("Number of unique RefSeq / Uniprot nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT', 'uniprot_id']).shape[0])
print("Number of unique RefSeq / Uniprot amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA', 'uniprot_id']).shape[0])

Number of rows: 3521
Number of unique RefSeq nucleotide mutations: 2875
Number of unique RefSeq amino acid mutations: 2873 <--
Number of unique RefSeq / Uniprot nucleotide mutations: 3521
Number of unique RefSeq / Uniprot amino acid mutations: 3519


Lost **~70** more amino acid mutations because the RefSeq mutation did not match the Uniprot sequence

In [30]:
mutation_df_3 = mutation_df.copy()

In [31]:
failed_3 = (
    set(mutation_df_2['uniprot_id'] + '.' + mutation_df_2['refseq_mutation']) - 
    set(mutation_df_3['uniprot_id'] + '.' + mutation_df_3['refseq_mutation'])
)

mutation_df_failed.loc[mutation_df_failed['unique_id'].isin(failed_3), 'errors'] = (
    mutation_df_failed.loc[mutation_df_failed['unique_id'].isin(failed_3), 'errors'] + 
    "RefSeq protein and UniProt protein do not have the same sequence. "
)

## Protein has domain(s)

Select only those mutations that fall inside a protein domain.

### mutation_df_4

In [32]:
mutation_df = mutation_df_3.copy()

In [33]:
sql_query = """\
select uniprot_domain_id, uniprot_id, domain_def, model_domain_def
from elaspic.uniprot_domain
join elaspic.uniprot_domain_template using (uniprot_domain_id)
left join elaspic.uniprot_domain_model using (uniprot_domain_id)
where uniprot_id in ('{}');
""".format("', '".join(set(mutation_df['uniprot_id'].values)))
uniprot_domain_model = pd.read_sql_query(sql_query, db_remote.engine)

In [34]:
mutation_df = (
    mutation_df
    .merge(uniprot_domain_model, on=['uniprot_id'])
)

In [35]:
mutation_df['mutation_in_domain'] = (
    mutation_df[['refseq_mutation', 'model_domain_def']]
        .apply(lambda x: ascommon.sequence_tools.mutation_in_domain(*x), axis=1)
)

In [36]:
mutation_df.loc[mutation_df['mutation_in_domain'].isnull(), 'mutation_in_domain'] = (
    mutation_df.loc[mutation_df['mutation_in_domain'].isnull(), ['refseq_mutation', 'domain_def']]
        .apply(lambda x: ascommon.sequence_tools.mutation_in_domain(*x), axis=1)
)

In [37]:
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0], "<--")
print("Number of unique RefSeq / Uniprot nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT', 'uniprot_id']).shape[0])
print("Number of unique RefSeq / Uniprot amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA', 'uniprot_id']).shape[0])

Number of rows: 4872
Number of unique RefSeq nucleotide mutations: 2524
Number of unique RefSeq amino acid mutations: 2523 <--
Number of unique RefSeq / Uniprot nucleotide mutations: 3103
Number of unique RefSeq / Uniprot amino acid mutations: 3102


More than **~300** additional mutations don't fall inside a protein with *any* structural domains.

In [38]:
mutation_df_4 = mutation_df.copy()

In [39]:
failed_4 = (
    set(mutation_df_3['uniprot_id'] + '.' + mutation_df_3['refseq_mutation']) - 
    set(mutation_df_4['uniprot_id'] + '.' + mutation_df_4['refseq_mutation'])
)

mutation_df_failed.loc[mutation_df_failed['unique_id'].isin(failed_4), 'errors'] = (
    mutation_df_failed.loc[mutation_df_failed['unique_id'].isin(failed_4), 'errors'] + 
    "Protein has no structural domains. "
)

## Mutation in domain

### mutation_df_5

In [40]:
mutation_df = mutation_df_4.copy()

In [41]:
mutation_df = mutation_df[mutation_df['mutation_in_domain']]

In [42]:
print("Number of rows:", 
    mutation_df.shape[0])
print("Number of unique RefSeq nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])
print("Number of unique RefSeq amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA']).shape[0], "<--")
print("Number of unique RefSeq / Uniprot nucleotide mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_NT', 'uniprot_id']).shape[0])
print("Number of unique RefSeq / Uniprot amino acid mutations:", 
    mutation_df.drop_duplicates(subset=['Mutation_RefSeq_AA', 'uniprot_id']).shape[0])

Number of rows: 2407
Number of unique RefSeq nucleotide mutations: 1965
Number of unique RefSeq amino acid mutations: 1964 <--
Number of unique RefSeq / Uniprot nucleotide mutations: 2407
Number of unique RefSeq / Uniprot amino acid mutations: 2406


Another **~550** mutations don't fall inside a domain for which we have a structural model (mostly because there is no structural domain in that region, but also because we didn't get around to making that homology model yet).

In [43]:
mutation_df_5 =  mutation_df.copy()

In [44]:
failed_5 = (
    set(mutation_df_4['uniprot_id'] + '.' + mutation_df_4['refseq_mutation']) - 
    set(mutation_df_5['uniprot_id'] + '.' + mutation_df_5['refseq_mutation'])
)

mutation_df_failed.loc[mutation_df_failed['unique_id'].isin(failed_5), 'errors'] = (
    mutation_df_failed.loc[mutation_df_failed['unique_id'].isin(failed_5), 'errors'] + 
    "Mutation falls outside of all structural domains. "
)

## ELASPIC

This is just to create a spreadsheet for Taipale.

### Load ELASPIC mutations

In [45]:
sql_query = """
select *
from elaspic.uniprot_domain_mutation
where (uniprot_id, mutation) in ({})
""".format(
    ', '.join(mutation_df_5[['uniprot_id', 'refseq_mutation']]
        .apply(
            lambda x: "('{}', '{}')".format(*x), 
            axis=1)))

uniprot_domain_mutation = pd.read_sql_query(sql_query, db_remote.engine)

In [46]:
display(uniprot_domain_mutation.head())
uniprot_domain_mutation.shape

,uniprot_id,uniprot_domain_id,mutation,mutation_errors,model_filename_wt,model_filename_mut,chain_modeller,mutation_modeller,stability_energy_wt,stability_energy_mut,physchem_wt,physchem_wt_ownchain,physchem_mut,physchem_mut_ownchain,matrix_score,secondary_structure_wt,solvent_accessibility_wt,secondary_structure_mut,solvent_accessibility_mut,provean_score,ddg,mut_date_modified
0,A0JLS0,125584718,R288Q,None,A0JLS0_R288Q/WT_RepairPDB_A0JLS0_4k0vB_1.pdb,A0JLS0_R288Q/MUT_RepairPDB_A0JLS0_4k0vB_1.pdb,A,R56Q,"231.344,-50.5828,-53.2911,-175.13,-4.2952,251....","230.334,-50.1268,-54.5928,-174.587,-3.65515,25...","0,0,0,0","0,0,4,38","0,0,0,0","0,0,4,38",1.0,T,20.88420,T,25.3845,-3.732,0.721349,2015-07-02 03:25:51
1,A0N0X8,95976016,I462V,None,A0N0X8_I462V/WT_RepairPDB_A0N0X8_4i8vD_1.pdb,A0N0X8_I462V/MUT_RepairPDB_A0N0X8_4i8vD_1.pdb,A,I426V,"55.4693,-325.792,-133.796,-596.836,-19.6376,78...","56.5188,-325.792,-133.796,-596.207,-19.6371,78...","0,0,0,0","0,0,0,49","0,0,0,0","0,0,0,49",3.0,H,2.67125,H,2.8616,-0.754,0.462089,2015-07-02 03:33:52
2,A0N0X8,95976016,T461N,None,A0N0X8_T461N/WT_RepairPDB_A0N0X8_4i8vD_1.pdb,A0N0X8_T461N/MUT_RepairPDB_A0N0X8_4i8vD_1.pdb,A,T425N,"55.4693,-325.792,-133.796,-596.836,-19.6376,78...","55.7297,-325.802,-133.796,-596.833,-19.6581,78...","0,0,0,0","0,0,0,25","0,0,0,0","0,0,0,25",0.0,H,50.81100,H,59.0385,-1.859,1.409460,2015-07-02 03:23:36
3,A2TDB8,33429238,L44P,None,A2TDB8_L44P/WT_RepairPDB_A2TDB8_2o10A_1.pdb,A2TDB8_L44P/MUT_RepairPDB_A2TDB8_2o10A_1.pdb,A,L38P,"23.4546,-25.0971,-16.5617,-57.5494,-1.19177,10...","27.3987,-25.0494,-16.5403,-56.2206,-1.19573,10...","0,0,0,0","0,0,0,43","0,0,0,0","0,0,0,43",-3.0,C,8.36953,C,11.7955,-6.883,2.629110,2015-07-02 03:17:54
4,A2TDC0,82143361,R18Q,None,A2TDC0_R18Q/WT_RepairPDB_A2TDC0_1ya5T_1.pdb,A2TDC0_R18Q/MUT_RepairPDB_A2TDC0_1ya5T_1.pdb,A,R18Q,"36.7197,-48.5438,-22.5117,-70.1828,-1.3093,96....","37.1465,-48.2814,-22.2494,-70.076,-1.21892,96....","0,0,0,0","0,0,1,12","0,0,0,0","0,0,1,12",1.0,T,93.70320,T,91.2993,-2.227,-0.203863,2015-07-02 03:17:45


(2401, 22)

In [47]:
mutation_df_failed.head()

,Mutation_RefSeq_AA,uniprot_id,refseq_mutation,unique_id,errors
0,NP_000005:p.C972Y,P01023,C972Y,P01023.C972Y,
1,NP_000005:p.V1000I,P01023,V1000I,P01023.V1000I,RefSeq protein and UniProt protein do not have...
2,NP_059132:p.C96Y,Q9NPC4,C96Y,Q9NPC4.C96Y,Protein has no structural domains.
3,NP_059132:p.S100L,Q9NPC4,S100L,Q9NPC4.S100L,Protein has no structural domains.
4,NP_059132:p.M183K,Q9NPC4,M183K,Q9NPC4.M183K,Protein has no structural domains.


In [48]:
nonfailed = set(
    mutation_df_failed[mutation_df_failed['errors'] == '']['uniprot_id'] + '.' +
    mutation_df_failed[mutation_df_failed['errors'] == '']['refseq_mutation']
)
elaspic_mutations = set(uniprot_domain_mutation['uniprot_id'] + '.' + uniprot_domain_mutation['mutation'])

In [49]:
failed_6 = set(mutation_df_5['uniprot_id'] + '.' + mutation_df_5['refseq_mutation'])
failed_6 = set(x for x in failed_6 if x not in elaspic_mutations and x in nonfailed)

In [50]:
print2("Unexplaind failures:", len(failed_6))
# 37

Unexplaind failures:                                        5


### Calculate missing

### Iterate on the above


    ...

### mutation_df_failed

In [51]:
mutation_df_failed.loc[
    (mutation_df_failed['uniprot_id'] + '.' + mutation_df_failed['refseq_mutation']).isin(failed_6), 
    'errors'
] = (
    "Error making homology model (probably low sequence identity). "
)

In [52]:
assert not set(mutation_df_failed['Mutation_RefSeq_AA']) ^ set(mutation_df_1['Mutation_RefSeq_AA'])

# Load histone interaction data

### mmc2_s2a

In [53]:
mmc2_s2a = pd.read_excel('../downloads/taipale/mmc2.xlsx', 'Table S2A')

In [54]:
display(mmc2_s2a.head(2))
print(mmc2_s2a.shape[0])
print(mmc2_s2a.drop_duplicates(subset=['Mutation_RefSeq_NT']).shape[0])

assert not set(mmc2_s2a['Mutation_RefSeq_NT']) - set(mutation_df_1['Mutation_RefSeq_NT'])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,WT_ELISA_score,Mut_ELISA_score,Both_expressed,WT_interaction_score,Mut_interaction_score,Differential_Z_score,Interact,Quality_control_factor_common,Quality_control_factor_official
0,Disease mutation,A4GALT,53947,53947_1850,NM_017436:c.287G>A,NP_059132:p.C96Y,0.51,0.79,1,1.398952,4.766989,3.105452,yes,BAG2,BAG2
1,Disease mutation,A4GALT,53947,53947_1852,NM_017436:c.299C>T,NP_059132:p.S100L,0.51,0.71,1,1.398952,5.029331,3.347341,yes,BAG2,BAG2


16464
2402


In [55]:
mmc2_s2a.drop(pd.Index(['Category', 'Symbol', 'Entrez_Gene_ID', 'Allele_ID', 'Mutation_RefSeq_AA']), axis=1, inplace=True)

# Load FoldX predictions

### mmc2_s2b

In [56]:
mmc2_s2b = pd.read_excel('../downloads/taipale/mmc2.xlsx', 'Table S2B')

In [57]:
display(mmc2_s2b.head(2))

assert not set(mmc2_s2b['Mutation_RefSeq_NT']) - set(mutation_df_1['Mutation_RefSeq_NT'])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,FoldX_value
0,Disease mutation,ACAT1,38,38_16884,NM_000019:c.218A>C,NP_000010:p.Q73P,7.841320
1,Disease mutation,ACAT1,38,38_16885,NM_000019:c.278A>G,NP_000010:p.N93S,0.236651


In [58]:
mmc2_s2b.drop(pd.Index(['Category', 'Symbol', 'Entrez_Gene_ID', 'Allele_ID', 'Mutation_RefSeq_AA']), axis=1, inplace=True)

# Save

### mutation_df_5

In [59]:
mutation_df = mutation_df_5.copy()

In [60]:
assert mutation_df['mutation_matches_sequence'].all()
assert mutation_df['mutation_in_domain'].all()

In [61]:
mutation_df[['uniprot_id', 'refseq_mutation']].head()

,uniprot_id,refseq_mutation
2,P01023,C972Y
6,Q9NRG9,S263P
7,Q9NRG9,L381R
8,Q9NRG9,L430F
9,F1T0I5,A129T


In [62]:
mutation_df.columns = datapkg.format_columns(mutation_df.columns)

In [63]:
display(mutation_df.head(2))
print(mutation_df.shape[0])

,category,symbol,entrez_gene_id,allele_id,mutation_refseq_nt,mutation_refseq_aa,hgmd_accession,hgmd_variant_class,db_snp_id,disease,refseq_id,refseq_mutation,uniprot_id,id,identifier_id,identifier_type,db,uniprot_name,protein_name,organism_name,gene_name,protein_existence,sequence_version,uniprot_sequence,mutation_matches_sequence,uniprot_domain_id,domain_def,model_domain_def,mutation_in_domain
2,Disease mutation,A2M,2,2_18118,NM_000014:c.2915G>A,NP_000005:p.C972Y,CM920001,DM,rs1800433,Chronic obstructive pulmonary disease,NP_000005,C972Y,P01023,61589,NP_000005.2,RefSeq,sp,A2MG_HUMAN,Alpha-2-macroglobulin,Homo sapiens,A2M,1.0,3.0,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,True,52474282,958:1270,958:1270,True
6,Disease mutation,AAAS,8086,8086_5619,NM_015665:c.787T>C,NP_056480:p.S263P,CM010150,DM,rs121918550,Triple-A syndrome,NP_056480,S263P,Q9NRG9,798718245,NP_056480.1,RefSeq,sp,AAAS_HUMAN,Aladin,Homo sapiens,AAAS,1.0,1.0,MCSLGLFPPPPPRGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...,True,108303701,122:487,130:487,True


2407


In [64]:
columns_to_keep = [
    'category', 'symbol', 'entrez_gene_id', 'allele_id', 
    'mutation_refseq_nt', 'mutation_refseq_aa', 'hgmd_accession', 'hgmd_variant_class', 'db_snp_id', 'disease', 
    # RefSeq info (processed)
    'refseq_id', 'refseq_mutation', 
    # UniProt info
    'uniprot_id', 'uniprot_domain_id', 'model_domain_def'
]

In [65]:
mutation_df[columns_to_keep].head(2)

,category,symbol,entrez_gene_id,allele_id,mutation_refseq_nt,mutation_refseq_aa,hgmd_accession,hgmd_variant_class,db_snp_id,disease,refseq_id,refseq_mutation,uniprot_id,uniprot_domain_id,model_domain_def
2,Disease mutation,A2M,2,2_18118,NM_000014:c.2915G>A,NP_000005:p.C972Y,CM920001,DM,rs1800433,Chronic obstructive pulmonary disease,NP_000005,C972Y,P01023,52474282,958:1270
6,Disease mutation,AAAS,8086,8086_5619,NM_015665:c.787T>C,NP_056480:p.S263P,CM010150,DM,rs121918550,Triple-A syndrome,NP_056480,S263P,Q9NRG9,108303701,130:487


In [66]:
print(mutation_df.shape[0])
print(mutation_df.drop_duplicates(subset=['mutation_refseq_nt']).shape[0])
print('RefSeq...')
print(mutation_df.drop_duplicates(subset=['refseq_id', 'refseq_mutation']).shape[0])
print(mutation_df.drop_duplicates(subset=['refseq_id', 'mutation_refseq_nt']).shape[0])
print('Uniprot...')
print(mutation_df.drop_duplicates(subset=['uniprot_id', 'refseq_mutation']).shape[0])
print(mutation_df.drop_duplicates(subset=['uniprot_id', 'mutation_refseq_nt']).shape[0])

2407
1965
RefSeq...
1964
1965
Uniprot...
2406
2407


In [67]:
t = db.import_df(
    mutation_df[columns_to_keep], 
    'taipale_tr',
)

In [68]:
db.add_idx_column(t.name)

2407

In [69]:
db.create_indexes(
    t.name, 
    [
        [('refseq_id', 'refseq_mutation'), False],
        [('uniprot_id', 'refseq_mutation'), False],
        [('uniprot_id', 'mutation_refseq_nt'), True],
        [('uniprot_domain_id', 'refseq_mutation'), False],
        (['mutation_refseq_nt'], False),
        (['mutation_refseq_aa'], False),
    ]
)

### mmc2_s2a

In [70]:
df = mmc2_s2a.copy()

In [71]:
df.columns = datapkg.format_columns(df.columns)

In [72]:
df.head()

,mutation_refseq_nt,wt_elisa_score,mut_elisa_score,both_expressed,wt_interaction_score,mut_interaction_score,differential_z_score,interact,quality_control_factor_common,quality_control_factor_official
0,NM_017436:c.287G>A,0.51,0.79,1,1.398952,4.766989,3.105452,yes,BAG2,BAG2
1,NM_017436:c.299C>T,0.51,0.71,1,1.398952,5.029331,3.347341,yes,BAG2,BAG2
2,NM_017436:c.548T>A,0.51,0.48,1,1.398952,1.595533,0.181255,NaN,BAG2,BAG2
3,NM_017436:c.656C>T,0.51,0.75,1,1.398952,4.833744,3.167002,yes,BAG2,BAG2
4,NM_015665:c.43C>A,0.20,0.35,0,6.765101,9.823864,2.820292,NaN,BAG2,BAG2


In [73]:
df['elisa_score_diff'] = df['mut_elisa_score'] - df['wt_elisa_score']
df['interaction_score_diff'] = df['mut_interaction_score'] - df['wt_interaction_score']

In [74]:
df.head()

,mutation_refseq_nt,wt_elisa_score,mut_elisa_score,both_expressed,wt_interaction_score,mut_interaction_score,differential_z_score,interact,quality_control_factor_common,quality_control_factor_official,elisa_score_diff,interaction_score_diff
0,NM_017436:c.287G>A,0.51,0.79,1,1.398952,4.766989,3.105452,yes,BAG2,BAG2,0.28,3.368036
1,NM_017436:c.299C>T,0.51,0.71,1,1.398952,5.029331,3.347341,yes,BAG2,BAG2,0.20,3.630378
2,NM_017436:c.548T>A,0.51,0.48,1,1.398952,1.595533,0.181255,NaN,BAG2,BAG2,-0.03,0.196581
3,NM_017436:c.656C>T,0.51,0.75,1,1.398952,4.833744,3.167002,yes,BAG2,BAG2,0.24,3.434791
4,NM_015665:c.43C>A,0.20,0.35,0,6.765101,9.823864,2.820292,NaN,BAG2,BAG2,0.15,3.058764


In [75]:
t = db.import_df(
    df,
    'taipale_tr_chaperone',
)

In [76]:
db.add_idx_column(t.name)

16464

In [77]:
db.create_indexes(
    t.name, 
    [
        (['mutation_refseq_nt'], False),
    ]
)

### mmc2_s2b

In [78]:
df = mmc2_s2b.copy()

In [79]:
df.columns = datapkg.format_columns(df.columns)

In [80]:
df.head()

,mutation_refseq_nt,fold_x_value
0,NM_000019:c.218A>C,7.841320
1,NM_000019:c.278A>G,0.236651
2,NM_000019:c.380C>T,0.215789
3,NM_000019:c.395C>G,1.050880
4,NM_005891:c.761T>C,2.247490


In [81]:
df.head()

,mutation_refseq_nt,fold_x_value
0,NM_000019:c.218A>C,7.841320
1,NM_000019:c.278A>G,0.236651
2,NM_000019:c.380C>T,0.215789
3,NM_000019:c.395C>G,1.050880
4,NM_005891:c.761T>C,2.247490


In [82]:
t = db.import_df(
    df, 
    'taipale_tr_foldx',
)

In [83]:
db.add_idx_column(t.name)

333

In [84]:
db.create_indexes(
    t.name, 
    [
        (['mutation_refseq_nt'], False),
    ]
)

## mutation_df_failed

In [85]:
df = mutation_df_failed.copy()

In [86]:
df.columns = datapkg.format_columns(df.columns)

In [87]:
df.head()

,mutation_refseq_aa,uniprot_id,refseq_mutation,unique_id,errors
0,NP_000005:p.C972Y,P01023,C972Y,P01023.C972Y,
1,NP_000005:p.V1000I,P01023,V1000I,P01023.V1000I,RefSeq protein and UniProt protein do not have...
2,NP_059132:p.C96Y,Q9NPC4,C96Y,Q9NPC4.C96Y,Protein has no structural domains.
3,NP_059132:p.S100L,Q9NPC4,S100L,Q9NPC4.S100L,Protein has no structural domains.
4,NP_059132:p.M183K,Q9NPC4,M183K,Q9NPC4.M183K,Protein has no structural domains.


In [88]:
t = db.import_df(
    df, 
    'taipale_tr_failed', 
)

In [89]:
db.add_idx_column(t.name)

3914

In [90]:
db.create_indexes(
    t.name, 
    [
        (['mutation_refseq_aa'], False),
    ]
)